In [1]:
import pandas as pd

In [2]:
train_path="../data/train.csv"
train_df = pd.read_csv(train_path)
print(train_df.head())

            ID                                              input  \
0  TRAIN_00000  별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈...   
1  TRAIN_00001                             잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ   
2  TRAIN_00002                                    절테 간면 않 된는 굣 멥몫   
3  TRAIN_00003  야... 칵컥 좋꾜 부됴 뼝 뚫렷썹 신원햐쥠만 닮패 넴센 밌쪄벅림. 샥퀘 핥류만 묵...   
4  TRAIN_00004  집윈 축쳐눌료 딴너왓눈뎁 카셩뷔 좋곱 칼쿰한네올. 쩌럼한뒈 뮬콰 욺료토 잊쿄 빻토 ...   

                                              output  
0  별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자...  
1                             잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ  
2                                    절대 가면 안 되는 곳 메모  
3  아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...  
4  지인 추천으로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...  


- 모델의 System Input으로 "복원: {난독화된 리뷰}" 형식으로 변환하기

In [3]:
def preprocess_data(dataframe, max_input_length=128, max_target_length=128):
    dataframe = dataframe.copy()
    dataframe['input_text'] = dataframe['input'].apply(lambda x: f"복원: {x}")
    dataframe['target_text'] = dataframe['output']
    dataframe = dataframe[['input_text', 'target_text']]
    return dataframe

# Preprocess the training data
preprocessed_train_df = preprocess_data(train_df)
print(preprocessed_train_df.head())

                                          input_text  \
0  복원: 별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 ...   
1                         복원: 잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ   
2                                복원: 절테 간면 않 된는 굣 멥몫   
3  복원: 야... 칵컥 좋꾜 부됴 뼝 뚫렷썹 신원햐쥠만 닮패 넴센 밌쪄벅림. 샥퀘 핥...   
4  복원: 집윈 축쳐눌료 딴너왓눈뎁 카셩뷔 좋곱 칼쿰한네올. 쩌럼한뒈 뮬콰 욺료토 잊쿄...   

                                         target_text  
0  별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자...  
1                             잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ  
2                                    절대 가면 안 되는 곳 메모  
3  아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...  
4  지인 추천으로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...  


In [ ]:
#2. 데이터 로드
train_df = pd.read_csv('../data/train.csv', encoding='utf-8-sig')
print("Train data shape:", train_df.shape)
print("\nSample data:")
display(train_df.head())

In [ ]:
# 5. 토크나이저 로드 및 분석
def analyze_tokenizer(tokenizer, sample_texts):
    """토크나이저 분석 함수"""
    print(f"Tokenizer 기본 정보:")
    print(f"Vocab size: {tokenizer.vocab_size}")
    print(f"Model max length: {tokenizer.model_max_length}")
    print(f"PAD token: {tokenizer.pad_token}, {tokenizer.pad_token_id}")
    print(f"EOS token: {tokenizer.eos_token}, {tokenizer.eos_token_id}")
    print(f"BOS token: {tokenizer.bos_token}, {tokenizer.bos_token_id}")
    
    print("\n샘플 텍스트 토큰화 예시:")
    for text in sample_texts:
        tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        decoded = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
        print(f"\n입력 텍스트: {text}")
        print(f"토큰화 결과: {decoded}")
        print(f"토큰 수: {len(decoded)}")

In [ ]:
# 샘플 텍스트로 토크나이저 분석
sample_texts = [
    train_data['input_text'].iloc[0],
    train_data['target_text'].iloc[0]
]
analyze_tokenizer(tokenizer, sample_texts)

In [4]:
import matplotlib.pyplot as plt
# 토큰 길이 계산 함수
def calculate_token_lengths(texts, tokenizer):
    token_lengths = [len(tokenizer(text, truncation=False)['input_ids']) for text in texts]
    return token_lengths

# 입력 텍스트와 타겟 텍스트 길이 계산
input_lengths = calculate_token_lengths(preprocessed_train_df['input_text'], tokenizer)
target_lengths = calculate_token_lengths(preprocessed_train_df['target_text'], tokenizer)

# 길이 분포 시각화
plt.hist(input_lengths, bins=50, alpha=0.6, label='Input Text Lengths')
plt.hist(target_lengths, bins=50, alpha=0.6, label='Target Text Lengths')
plt.axvline(x=int(pd.Series(input_lengths).quantile(0.99)), color='blue', linestyle='--', label='95% Input')
plt.axvline(x=int(pd.Series(target_lengths).quantile(0.99)), color='orange', linestyle='--', label='95% Target')
plt.xlabel('Token Length')
plt.ylabel('Frequency')
plt.title('Token Length Distribution')
plt.legend()
plt.show()

NameError: name 'tokenizer' is not defined

In [ ]:
# 95% 커버 기준 max_length 설정
input_max_length = int(pd.Series(input_lengths).quantile(0.99))
target_max_length = int(pd.Series(target_lengths).quantile(0.99))
print(f"99% 기준 Input Max Length: {input_max_length}")
print(f"99% 기준 Target Max Length: {target_max_length}")